In [1]:
import os
import imageio
import sys
import random
import numpy as np
import pandas as pd
from keras.models import Model
from keras.applications import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import GlobalAveragePooling2D, Input
from skimage.transform import resize
from imgaug import augmenters as iaa
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
import tensorflow as tf

target_size = (299, 299)
batch_size = 256
epochs = 1000
dropout_rate = 0.5
learning_rate = 0.0001

work_dir = '/home/obsk/workspace/ai_osteoporosis_hip'
share_dir = '/share/ai_osteoporosis_hip'
crop_dir = os.path.join(share_dir, 'crop')
xls_dir = os.path.join(share_dir, 'excel_files')
xls_final = os.path.join(xls_dir, 'final.xlsx')
df_final = pd.read_excel(xls_final)

conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(target_size[0], target_size[1], 3))

i = conv_base.input
x = conv_base(i)
x = GlobalAveragePooling2D()(x)
model = Model(inputs=i, outputs=x)

/home/obsk/anaconda3/envs/naesa1.8/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
inception_v3 (Model)         (None, 8, 8, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________


In [17]:
import random
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5, 5) 

# Augmentation parameters
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
#    iaa.Flipud(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Multiply((0.9, 1.1)),
    iaa.Affine(
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, #{"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, 
        rotate=(-40, 40),
        shear=(-20, 20),
        order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
        cval=0,
        mode='edge'#['constant', 'edge']
    ),
    iaa.OneOf([
        iaa.GaussianBlur((0, 1.0)),
#        iaa.AverageBlur(k=(2, 7)),
        iaa.Sharpen(alpha=(0, 0.1), lightness=(0.75, 1.5))
    ]),
])


def generate_df(xls_file):
    df = pd.read_excel(xls_file, dtype={'patient_id':str})

    d1 = df[df['split']==1]

    columns_list = ['age', 'height', 'weight', 'bmi']
    
    for column_name in columns_list:
        mean = d1[column_name].mean()
        std = d1[column_name].std()
        df[column_name] = df[column_name].apply(lambda x: (x - mean) / std)
        
    sex_to_code = {'M': 1, 'F':0}
    df['sex_code'] = df['sex'].apply(lambda x: sex_to_code[x])    
    
    return df

def resample(df):
    resampled_list = []
    
    for i in range(len(df)):
        age = df.iloc[i]['age']
        sex = df.iloc[i]['sex_code']
        bmi = df.iloc[i]['bmi']
        class2_1 = df.iloc[i]['class2_1']
        split = df.iloc[i]['split']
        file_name = df.iloc[i]['file_name']
        if split in [2, 3]:
            resampled_list.append((age, sex, bmi, class2_1, split, file_name))
        elif class2_1 == 0:
            resampled_list.append((age, sex, bmi, class2_1, split, file_name))
        else:
            resampled_list.append((age, sex, bmi, class2_1, split, file_name))
            resampled_list.append((age, sex, bmi, class2_1, split, file_name))
            resampled_list.append((age, sex, bmi, class2_1, split, file_name))
    
    random.shuffle(resampled_list)
    df2 = pd.DataFrame(resampled_list, columns=['age', 'sex_code', 'bmi', 'class2_1', 'split', 'file_name'])
                     
    return df2

def read_image(file_path, target_size, is_train):
    img_gray = np.array(imageio.imread(file_path))
    max_val = np.amax(img_gray)
    img_gray = img_gray / max_val
    img_resized = resize(img_gray, target_size)
    img = np.dstack((img_resized, img_resized, img_resized))

    if is_train:
        #img = seq.augment_images(x)
        #plt.imshow(img, cmap='gray')
        #plt.show()
        img = seq.augment_image(img)
        #plt.imshow(img, cmap='gray')
        #plt.show()        
    return img

def extract_features(df, dir_name, target_size, batch_size, set_name):
    split_dict = {'train':1, 'validation':2, 'test':3}
    split = split_dict[set_name]
    data = df[df['split']==split]
    count = len(data)
    features = np.zeros(shape=(count*epochs, 2048))
    labels = np.zeros(shape=(count*epochs))
    
    div = count // batch_size
    res = count % batch_size
    
    i = 0
    
    print('sample count=%d' % count)

    if set_name == 'train':
        for l in range(epochs):
            i = 0
            for n in range(div+1):
                if n == div and res != 0: img_batch_size = res
                else: img_batch_size = batch_size
                img_batch = np.zeros(shape=(img_batch_size, target_size[0], target_size[1], 3))

                for m in range(img_batch_size):
                    file_name = data.iloc[i]['file_name']
                    file_path = os.path.join(dir_name, file_name)
                    class2_1 = data.iloc[i]['class2_1']

                    img_batch[m] = read_image(file_path, target_size, is_train=True)
                    labels[i] = class2_1
                    i += 1
                    print('Processing image %d...\r' % i, end='')
                    sys.stdout.flush()
                features_batch = model.predict(img_batch)
                features[count*l+n*batch_size : count*l+n*batch_size+img_batch_size] = features_batch            
                print('Filling batch %d~%d...' % (count*l+n*batch_size ,count*l+n*batch_size+img_batch_size))
    else:
        for n in range(div+1):
            if n == div and res != 0: img_batch_size = res
            else: img_batch_size = batch_size
            img_batch = np.zeros(shape=(img_batch_size, target_size[0], target_size[1], 3))

            for m in range(img_batch_size):
                file_name = data.iloc[i]['file_name']
                file_path = os.path.join(dir_name, file_name)
                class2_1 = data.iloc[i]['class2_1']

                img_batch[m] = read_image(file_path, target_size, is_train=False)
                labels[i] = class2_1
                i += 1
                print('Processing image %d...\r' % i, end='')
                sys.stdout.flush()
            features_batch = model.predict(img_batch)
            features[n*batch_size : n*batch_size+img_batch_size] = features_batch            
                
    return features, labels

In [5]:
df_final = generate_df(xls_final)
df_resample = resample(df_final)

In [ ]:
train_features, train_labels = extract_features(df_resample, crop_dir, target_size, batch_size, 'train')
validation_features, validation_labels = extract_features(df_resample, crop_dir, target_size, batch_size, 'validation')
test_features, test_labels = extract_features(df_resample, crop_dir, target_size, batch_size, 'test')

sample count=13052


/home/obsk/anaconda3/envs/naesa1.8/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/obsk/anaconda3/envs/naesa1.8/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Filling batch 0~256....
Filling batch 256~512...
Filling batch 512~768...
Filling batch 768~1024...
Filling batch 1024~1280...
Filling batch 1280~1536...
Filling batch 1536~1792...
Filling batch 1792~2048...
Filling batch 2048~2304...
Filling batch 2304~2560...
Filling batch 2560~2816...
Filling batch 2816~3072...
Filling batch 3072~3328...
Filling batch 3328~3584...
Filling batch 3584~3840...
Filling batch 3840~4096...
Filling batch 4096~4352...
Filling batch 4352~4608...
Filling batch 4608~4864...
Filling batch 4864~5120...
Filling batch 5120~5376...
Filling batch 5376~5632...
Filling batch 5632~5888...
Filling batch 5888~6144...
Filling batch 6144~6400...
Filling batch 6400~6656...
Filling batch 6656~6912...
Filling batch 6912~7168...
Filling batch 7168~7424...
Filling batch 7424~7680...
Filling batch 7680~7936...
Filling batch 7936~8192...
Filling batch 8192~8448...
Filling batch 8448~8704...
Filling batch 8704~8960...
Filling batch 8960~9216...
Filling batch 9216~9472...
Filling b

In [73]:
from keras import models
from keras import layers
from keras import optimizers

model2 = models.Sequential()
model2.add(layers.Dense(1024, activation='relu', input_dim=2048))
model2.add(layers.Dropout(dropout_rate))
model2.add(layers.Dense(1, activation='sigmoid'))

model2.compile(optimizer=optimizers.RMSprop(learning_rate),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model2.fit(train_features, train_labels,
                    epochs=1000,
                    batch_size=batch_size,
                    validation_data=(validation_features, validation_labels))

Train on 13052 samples, validate on 2330 samples
Epoch 1/1000
13052/13052 [==============================] - 19s 1ms/step - loss: 0.6256 - acc: 0.6652 - val_loss: 0.3676 - val_acc: 0.8519
Epoch 2/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.5188 - acc: 0.7436 - val_loss: 0.3817 - val_acc: 0.8296
Epoch 3/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.4871 - acc: 0.7629 - val_loss: 0.5121 - val_acc: 0.7567
Epoch 4/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.4726 - acc: 0.7750 - val_loss: 0.3491 - val_acc: 0.8459
Epoch 5/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.4542 - acc: 0.7826 - val_loss: 0.4034 - val_acc: 0.8129
Epoch 6/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.4491 - acc: 0.7861 - val_loss: 0.5300 - val_acc: 0.7369
Epoch 7/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.4427 - acc: 0.7918 - val_loss: 0.4

13052/13052 [==============================] - 1s 43us/step - loss: 0.2683 - acc: 0.8915 - val_loss: 0.2989 - val_acc: 0.8652
Epoch 60/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.2620 - acc: 0.8913 - val_loss: 0.3160 - val_acc: 0.8609
Epoch 61/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.2614 - acc: 0.8918 - val_loss: 0.3762 - val_acc: 0.8399
Epoch 62/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.2596 - acc: 0.8947 - val_loss: 0.3021 - val_acc: 0.8670
Epoch 63/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.2588 - acc: 0.8920 - val_loss: 0.3137 - val_acc: 0.8627
Epoch 64/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.2572 - acc: 0.8939 - val_loss: 0.4579 - val_acc: 0.7923
Epoch 65/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.2500 - acc: 0.8978 - val_loss: 0.3430 - val_acc: 0.8502
Epoch 66/1000
13052/13052 [=======

13052/13052 [==============================] - 1s 43us/step - loss: 0.1591 - acc: 0.9435 - val_loss: 0.3426 - val_acc: 0.8571
Epoch 118/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.1596 - acc: 0.9428 - val_loss: 0.3335 - val_acc: 0.8601
Epoch 119/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.1671 - acc: 0.9378 - val_loss: 0.3675 - val_acc: 0.8455
Epoch 120/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.1562 - acc: 0.9429 - val_loss: 0.4648 - val_acc: 0.8030
Epoch 121/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.1597 - acc: 0.9424 - val_loss: 0.3486 - val_acc: 0.8549
Epoch 122/1000
13052/13052 [==============================] - 1s 44us/step - loss: 0.1580 - acc: 0.9426 - val_loss: 0.3208 - val_acc: 0.8674
Epoch 123/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.1542 - acc: 0.9458 - val_loss: 0.3541 - val_acc: 0.8515
Epoch 124/1000
13052/13052 [

13052/13052 [==============================] - 1s 43us/step - loss: 0.0949 - acc: 0.9694 - val_loss: 0.3591 - val_acc: 0.8734
Epoch 176/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0929 - acc: 0.9716 - val_loss: 0.5464 - val_acc: 0.7944
Epoch 177/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0960 - acc: 0.9685 - val_loss: 0.3887 - val_acc: 0.8691
Epoch 178/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0898 - acc: 0.9722 - val_loss: 0.4062 - val_acc: 0.8494
Epoch 179/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0909 - acc: 0.9707 - val_loss: 0.4877 - val_acc: 0.8210
Epoch 180/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0981 - acc: 0.9670 - val_loss: 0.3658 - val_acc: 0.8618
Epoch 181/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0928 - acc: 0.9715 - val_loss: 0.3821 - val_acc: 0.8554
Epoch 182/1000
13052/13052 [

13052/13052 [==============================] - 1s 43us/step - loss: 0.0509 - acc: 0.9871 - val_loss: 0.4072 - val_acc: 0.8592
Epoch 234/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0568 - acc: 0.9820 - val_loss: 0.4050 - val_acc: 0.8665
Epoch 235/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0558 - acc: 0.9844 - val_loss: 0.4881 - val_acc: 0.8373
Epoch 236/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0556 - acc: 0.9856 - val_loss: 0.4058 - val_acc: 0.8678
Epoch 237/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0608 - acc: 0.9814 - val_loss: 0.4056 - val_acc: 0.8532
Epoch 238/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0631 - acc: 0.9817 - val_loss: 0.4189 - val_acc: 0.8494
Epoch 239/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0441 - acc: 0.9907 - val_loss: 0.4343 - val_acc: 0.8498
Epoch 240/1000
13052/13052 [

...

13052/13052 [==============================] - 1s 43us/step - loss: 0.0326 - acc: 0.9894 - val_loss: 0.4739 - val_acc: 0.8605
Epoch 350/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0265 - acc: 0.9929 - val_loss: 0.4837 - val_acc: 0.8584
Epoch 351/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0240 - acc: 0.9932 - val_loss: 0.4922 - val_acc: 0.8575
Epoch 352/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0306 - acc: 0.9894 - val_loss: 0.7436 - val_acc: 0.8687
Epoch 353/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0319 - acc: 0.9904 - val_loss: 0.4824 - val_acc: 0.8584
Epoch 354/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0267 - acc: 0.9920 - val_loss: 0.4886 - val_acc: 0.8571
Epoch 355/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0230 - acc: 0.9933 - val_loss: 0.5308 - val_acc: 0.8528
Epoch 356/1000
13052/13052 [

13052/13052 [==============================] - 1s 42us/step - loss: 0.0202 - acc: 0.9937 - val_loss: 0.4912 - val_acc: 0.8605
Epoch 408/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0273 - acc: 0.9910 - val_loss: 0.5078 - val_acc: 0.8592
Epoch 409/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0171 - acc: 0.9949 - val_loss: 0.5150 - val_acc: 0.8567
Epoch 410/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0226 - acc: 0.9934 - val_loss: 0.5228 - val_acc: 0.8584
Epoch 411/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0191 - acc: 0.9944 - val_loss: 0.5342 - val_acc: 0.8571
Epoch 412/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0224 - acc: 0.9932 - val_loss: 0.5265 - val_acc: 0.8592
Epoch 413/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0202 - acc: 0.9938 - val_loss: 0.5434 - val_acc: 0.8575
Epoch 414/1000
13052/13052 [

13052/13052 [==============================] - 1s 44us/step - loss: 0.0181 - acc: 0.9938 - val_loss: 0.5434 - val_acc: 0.8575
Epoch 466/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0157 - acc: 0.9942 - val_loss: 0.5478 - val_acc: 0.8601
Epoch 467/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0215 - acc: 0.9930 - val_loss: 0.5477 - val_acc: 0.8575
Epoch 468/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0190 - acc: 0.9933 - val_loss: 0.5556 - val_acc: 0.8558
Epoch 469/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0170 - acc: 0.9941 - val_loss: 0.6142 - val_acc: 0.8464
Epoch 470/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0194 - acc: 0.9938 - val_loss: 0.5480 - val_acc: 0.8609
Epoch 471/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0185 - acc: 0.9938 - val_loss: 0.5501 - val_acc: 0.8627
Epoch 472/1000
13052/13052 [

13052/13052 [==============================] - 1s 43us/step - loss: 0.0159 - acc: 0.9941 - val_loss: 0.5777 - val_acc: 0.8584
Epoch 524/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0181 - acc: 0.9941 - val_loss: 0.5569 - val_acc: 0.8622
Epoch 525/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0255 - acc: 0.9932 - val_loss: 0.5651 - val_acc: 0.8605
Epoch 526/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0278 - acc: 0.9910 - val_loss: 0.5719 - val_acc: 0.8592
Epoch 527/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0035 - acc: 0.9999 - val_loss: 0.6353 - val_acc: 0.8489
Epoch 528/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0175 - acc: 0.9936 - val_loss: 0.5976 - val_acc: 0.8536
Epoch 529/1000
13052/13052 [==============================] - 1s 43us/step - loss: 0.0163 - acc: 0.9953 - val_loss: 0.5900 - val_acc: 0.8562
Epoch 530/1000
13052/13052 [

KeyboardInterrupt: 

In [82]:

#df = get_clinical_data(csv_file)

#test_file_list, img_test, clinical_test, y_test = get_image_data(test_dir, df)
#test_file_list = []
#for path, dirs, files in os.walk(test_dir):
#    test_file_list += [os.path.join(path, file) for file in files]
#count = len(test_file_list)

#img_test = preprocess_input(img_test, is_train=False)
y_pred = model2.predict(test_features)
scores = model2.evaluate(test_features, test_labels_noaug, batch_size)

#scores = model.evaluate_generator(custom_generator(test_file_list, df, is_train=False, non_image_data=True), steps=math.ceil(test_set_size/batch_size))
print("%s: %.2f%%" %(model2.metrics_names[1], scores[1]*100))


fpr, tpr, thresholds = roc_curve(test_labels_noaug, y_pred)
auc = auc(fpr, tpr)
print('auc=%0.3f' % auc)

533/533 [==============================] - 0s 39us/step
acc: 84.05%
auc=0.869


In [85]:
for label, pred in zip(test_labels_noaug, y_pred):
    print('%d, %0.1f%%' % (label, pred*100))

0, 0.0%
0, 2.7%
0, 0.0%
0, 97.3%
0, 0.0%
0, 0.1%
0, 0.0%
0, 0.0%
0, 0.0%
0, 0.0%
0, 1.0%
0, 0.1%
0, 0.0%
0, 30.4%
0, 0.1%
1, 99.9%
0, 7.3%
0, 0.0%
0, 9.0%
0, 99.9%
0, 0.0%
1, 79.8%
1, 0.1%
0, 0.0%
0, 0.0%
0, 2.1%
0, 0.0%
1, 59.3%
0, 0.0%
0, 0.0%
0, 27.0%
0, 0.0%
0, 100.0%
1, 96.5%
0, 0.0%
0, 0.0%
0, 0.0%
0, 0.0%
0, 100.0%
0, 0.1%
0, 4.2%
0, 0.1%
0, 99.6%
0, 0.0%
1, 100.0%
0, 99.8%
0, 0.0%
0, 94.7%
0, 0.0%
0, 0.3%
0, 9.6%
1, 97.0%
0, 0.0%
0, 99.8%
0, 0.0%
0, 0.0%
1, 0.1%
0, 0.0%
0, 7.4%
0, 0.2%
1, 100.0%
1, 98.9%
0, 0.3%
0, 0.0%
0, 0.0%
0, 3.0%
0, 0.0%
0, 0.0%
0, 3.1%
1, 48.5%
0, 0.0%
0, 0.0%
0, 0.0%
0, 4.5%
0, 1.2%
0, 0.0%
0, 45.7%
0, 1.6%
0, 50.2%
0, 8.8%
0, 0.1%
0, 95.0%
0, 96.4%
0, 97.0%
0, 0.1%
1, 1.0%
0, 2.5%
0, 13.8%
0, 0.0%
0, 1.5%
0, 8.1%
0, 0.0%
0, 0.0%
0, 0.0%
0, 0.0%
0, 0.9%
0, 0.0%
1, 3.3%
1, 100.0%
1, 100.0%
0, 0.2%
0, 0.0%
1, 100.0%
0, 18.8%
0, 0.0%
0, 0.0%
0, 0.4%
0, 0.1%
0, 0.0%
0, 1.3%
1, 100.0%
0, 0.0%
0, 0.5%
0, 0.0%
1, 99.9%
0, 99.9%
0, 0.0%
0, 67.4%
0, 3.3%
1, 2.1%